# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [52]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("dataframe").getOrCreate() #builder, NOT Builder

In [53]:
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [2]:
path = "../data/pga_tour_historical.csv"
pga_tour = spark.read.csv(path,inferSchema=True,header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [3]:
pga_tour.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [4]:
pga_tour.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [5]:
pga_tour.printSchema() # Value - string?

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [13]:
from pyspark.sql.types import *
#gonna use [StructField("field_name",DataType,nullable),,,]   -> StructType   

field_schema = [StructField("Player Name",StringType(),True),
               StructField("Season",IntegerType(),True),
               StructField("Statistic",StringType(),True),
               StructField("Variable",StringType(),True),
               StructField("Value",IntegerType(),True)]

schema = StructType(fields=field_schema)

pga = spark.read.csv(path,schema=schema,header=True)

In [14]:
pga.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [15]:
pga.select("Value").describe().show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570787|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [16]:
pga.select("Value","Season").summary("min","max").show()

+-------+-------+------+
|summary|  Value|Season|
+-------+-------+------+
|    min|   -178|  2010|
|    max|3564954|  2018|
+-------+-------+------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [17]:
new_pga = pga.select("Season","Value")

new_pga.write.mode("overwrite").parquet("../data/with_out_partition")

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [18]:
new_pga.write.partitionBy("Season").mode("overwrite").parquet("../data/with_partition")

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [45]:
from pyspark.sql.functions import col
pga_parquet1 = spark.read.parquet("../data/with_partition")

pga_parquet1.filter(pga_parquet1.Value.isNotNull()).count()

1657247

In [46]:
pga_parquet1.filter(pga_parquet1.Value.isNull()).count()

1083156

## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [47]:
path = "../data/with_partition/Season="
pga_partitioned = spark.read.parquet(path+"2010", path+"2011",path+"2012")

In [55]:
pga_partitioned.show(5) 

#this gives you only Value!

+-----+
|Value|
+-----+
|   71|
|   77|
|   83|
|   54|
|  100|
+-----+
only showing top 5 rows



In [57]:
#We need to use a method to get the "Season" and "Value" Columns 

path = '../data/with_partition/'

dataframe = spark.read.option("basePath",path).parquet(path+"Season=2010", \
                                                      path + "Season=2011",
                                                      path + "Season=2012")

dataframe.show(5)

#remember, option!

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [50]:
#ID name password Email CreatedAt
import datetime

frame_schema = [StructField("ID",StringType(),False),
               StructField("Name",StringType(),False),
               StructField("PassWord",IntegerType(),False),
               StructField("Email",StringType(),False),
               StructField("CreatedAt",DateType(),False)]

schema = StructType(fields=frame_schema)

rows = [("saka1023","Migo",1234,"saka1023@naver.com",datetime.datetime.now()),
       ("kcee323","Mogo",5234,"coupang@naver.com",datetime.datetime.now()),
       ("sreta10","Mego",7234,"bigdata@naver.com",datetime.datetime.now()),
       ("3132sdd","Mago",1234,"engineer@naver.com",datetime.datetime.now())]

custom_df = spark.createDataFrame(rows,schema=schema)


In [51]:
custom_df.show()

+--------+----+--------+------------------+----------+
|      ID|Name|PassWord|             Email| CreatedAt|
+--------+----+--------+------------------+----------+
|saka1023|Migo|    1234|saka1023@naver.com|2021-09-03|
| kcee323|Mogo|    5234| coupang@naver.com|2021-09-03|
| sreta10|Mego|    7234| bigdata@naver.com|2021-09-03|
| 3132sdd|Mago|    1234|engineer@naver.com|2021-09-03|
+--------+----+--------+------------------+----------+



In [61]:
custom_df_without_schema = spark.createDataFrame(rows,custom_df.columns)

### Comparison...

In [63]:
custom_df.printSchema()

root
 |-- ID: string (nullable = false)
 |-- Name: string (nullable = false)
 |-- PassWord: integer (nullable = false)
 |-- Email: string (nullable = false)
 |-- CreatedAt: date (nullable = false)



In [62]:
custom_df_without_schema.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- PassWord: long (nullable = true)
 |-- Email: string (nullable = true)
 |-- CreatedAt: timestamp (nullable = true)



In [64]:
custom_df.show(3) #DateType

+--------+----+--------+------------------+----------+
|      ID|Name|PassWord|             Email| CreatedAt|
+--------+----+--------+------------------+----------+
|saka1023|Migo|    1234|saka1023@naver.com|2021-09-03|
| kcee323|Mogo|    5234| coupang@naver.com|2021-09-03|
| sreta10|Mego|    7234| bigdata@naver.com|2021-09-03|
+--------+----+--------+------------------+----------+
only showing top 3 rows



In [65]:
custom_df_without_schema.show(3) #TimestampType

+--------+----+--------+------------------+--------------------+
|      ID|Name|PassWord|             Email|           CreatedAt|
+--------+----+--------+------------------+--------------------+
|saka1023|Migo|    1234|saka1023@naver.com|2021-09-03 23:23:...|
| kcee323|Mogo|    5234| coupang@naver.com|2021-09-03 23:23:...|
| sreta10|Mego|    7234| bigdata@naver.com|2021-09-03 23:23:...|
+--------+----+--------+------------------+--------------------+
only showing top 3 rows



## We're done! Great job!